# Prepare the data

In [27]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [29]:
chars = sorted(list(set("".join(words))))
vocab_size = len(chars)

In [44]:
stoi = {char:i for i, char in enumerate(chars)}
stoi['.'] = 0 
itos = {i: char for char, i  in stoi.items()}

In [56]:
block_size = 3 
X, Y = [], []

for w in words:
    context = [0]*block_size
    for ch in w+'.':
        ix = stoi[ch]
        Y.append(ix)
        X.append(context)
        context = context[1:] + [ix]
        
X = torch.tensor(X)
Y = torch.tensor(Y)
len(chars)

27

# MLP

In [59]:
# create an embeding matrix
# 26 x 20 
E = torch.randn((27, 20), requires_grad=True)


torch.Size([27, 20])